# GPU programming in Python

## Short refresh on technical vocabulary

Several important terms in the topic of CUDA programming are listed here:
-  host: the CPU
-  device: the GPU
-  host memory: the system main memory
-  device memory: onboard memory on a GPU card
-  kernel: a GPU function launched by the host and executed on the device
-  device function: a GPU function executed on the device which can only be called from the device (i.e. from a kernel or another device function)


The typycal GPU structure is illustrated below

<img src="images/CUDA-threads.png" width="75%" height="75%" />

## Cupy

<img src="images/cupy.png" width="75%" height="75%" />

CuPy is a NumPy/SciPy-compatible array library for GPU-accelerated computing with Python. CuPy acts as a drop-in replacement to run existing NumPy/SciPy code on NVIDIA CUDA or AMD ROCm platforms.


### Installation

Before installing Cupy is recommended to install setuptools

```
$ python -m pip install -U setuptools pip
```

Then we can install cupy with conda, for a specific cuda version as

```
$ conda install -c conda-forge cupy cuda-version=11.8
```

I suggest to install also CUDNN and CUTENSOR
```
$ conda install -c conda-forge cupy cudnn cutensor nccl
```

In [1]:
import numpy as np
import cupy as cp

Let's create some arrays on CPU and GPU and compare time

In [6]:
#CPU
x_on_cpu = np.array([1, 2, 3, 4, 5])

%timeit -n 100 x_cpu = np.ones((100,500,500))


49 ms ± 2.73 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
#GPU
x_on_gpu = cp.array([1, 2, 3, 4, 5])

%timeit -n 100 x_gpu = cp.ones((100,500,500))




23.5 µs ± 13 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


**Data transfer**

*How do we move arrays from CPU to GPUs?*

In CuPy, all CUDA operations such as data transfer (see the Data Transfer section) and kernel launches are enqueued onto the current stream, and the queued tasks on the same stream will be executed in serial (but asynchronously with respect to the host).

The default current stream in CuPy is CUDA’s null stream (i.e., stream 0). It is also known as the legacy default stream, which is unique per device. However, it is possible to change the current stream using the cupy.cuda.Stream API, please see Accessing CUDA Functionalities for example. The current stream in CuPy can be retrieved using cupy.cuda.get_current_stream().



In [10]:
# FROM HOST TO DEVICE

x_cpu = np.array([1, 2, 3])
%timeit x_gpu = cp.asarray(x_cpu)  # move the data to the current device.

#VICEVERSA: FROM DEVICE TO HOST

x_gpu = cp.array([1, 2, 3])  # create an array in the current device
%timeit x_cpu = cp.asnumpy(x_gpu)  # move the array to the host.

#alternative:
%timeit x_cpu = x_gpu.get()

28.7 µs ± 943 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
18.5 µs ± 2.24 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)
17.5 µs ± 2.14 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


using the Device context manager:

In [11]:


with cp.cuda.Device(0):
  x_on_gpu0 = cp.array([1, 2, 3, 4, 5])

#with cp.cuda.Device(1):
#   x_on_gpu1 = cp.array([1, 2, 3, 4, 5])


## How to write CPU/GPU agnostic code
CuPy’s compatibility with NumPy makes it possible to write CPU/GPU agnostic code. For this purpose, CuPy implements the cupy.get_array_module() function that returns a reference to cupy if any of its arguments resides on a GPU and numpy otherwise. Here is an example of a CPU/GPU agnostic function that computes

In [ ]:
# Stable implementation of log(1 + exp(x))
def softplus(x):
    xp = cp.get_array_module(x)  # 'xp' is a standard usage in the community
    print("Using:", xp.__name__)
    return xp.maximum(0, x) + xp.log1p(xp.exp(-abs(x)))

x_cpu = np.random.random(1000)

%timeit softplus(x_cpu)

x_gpu = cp.random.random(1000)

%timeit softplus(x_gpu)

Let's compare the execution time on CPU and GPUs of 3 different linear algebra operations:
- matmul
- SVD (Singular Value Decomposition)
- trace

In [41]:
#Matmul on CPU
a = np.random.rand(1000,1000)
b = np.random.rand(1000,1000)

%timeit -n 10 np.matmul(a,b,out=None)

%timeit -n 10 np.trace(a, out=None)

58 ms ± 12.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
8.4 µs ± 4.39 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [40]:
#Matmul on GPU
a = cp.random.rand(1000,1000)
b = cp.random.rand(1000,1000)

%timeit -n 10 cp.matmul(a,b,out=None)


%timeit -n 10 cp.trace(a, out=None)

The slowest run took 4.49 times longer than the fastest. This could mean that an intermediate result is being cached.
349 µs ± 250 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
66.3 µs ± 7.05 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
#SVD on CPU
%%time
x_cpu = np.random.random((1000, 1000))
u, s, v = np.linalg.svd(x_cpu)


CPU times: user 1.37 s, sys: 285 ms, total: 1.66 s
Wall time: 895 ms


In [13]:
#SVD on GPU

%%time
x_gpu = cp.random.random((1000, 1000))
u, s, v = cp.linalg.svd(x_gpu)

CPU times: user 1.57 s, sys: 316 ms, total: 1.89 s
Wall time: 5.72 s


this is not

In [16]:
import time
start_gpu = cp.cuda.Event()
end_gpu = cp.cuda.Event()

start_gpu.record()
start_cpu = time.perf_counter()
x_gpu = cp.random.random((1000, 1000))
u, s, v = cp.linalg.svd(x_gpu)
end_cpu = time.perf_counter()
end_gpu.record()
end_gpu.synchronize()
t_gpu = cp.cuda.get_elapsed_time(start_gpu, end_gpu)
t_cpu = end_cpu - start_cpu

print("time measured with cuda event", t_gpu)
print("time measure with time", t_cpu)

time measured with cuda event 368.1689453125
time measure with time 0.36806521299968153


In [18]:
#CUPY PROFILER
from cupyx.profiler import benchmark
a = cp.random.random((256, 256, 256), dtype=cp.float32)
print(benchmark(a.sum, (), n_repeat=100))

sum                 :    CPU:   25.292 us   +/- 5.547 (min:   21.103 / max:   51.869) us     GPU-0:  101.437 us   +/- 5.273 (min:   96.256 / max:  125.952) us


## Memory management
CuPy uses memory pool for memory allocations by default. The memory pool significantly improves the performance by mitigating the overhead of memory allocation and CPU/GPU synchronization.

There are two different memory pools in CuPy:

   - Device memory pool (GPU device memory), which is used for GPU memory allocations.

   - Pinned memory pool (non-swappable CPU memory), which is used during CPU-to-GPU data transfer.


You can clear the memory pool by calling `free_all_blocks`.

```python
mempool.free_all_blocks()
```

In [15]:
import cupy
import numpy

mempool = cupy.get_default_memory_pool()
pinned_mempool = cupy.get_default_pinned_memory_pool()

# Create an array on CPU.
# NumPy allocates 400 bytes in CPU (not managed by CuPy memory pool).
a_cpu = numpy.ndarray(100, dtype=numpy.float32)
print(a_cpu.nbytes)                      # 400

# You can access statistics of these memory pools.
print(mempool.used_bytes())              # 0
print(mempool.total_bytes())             # 0
print(pinned_mempool.n_free_blocks())    # 0

# Transfer the array from CPU to GPU.
# This allocates 400 bytes from the device memory pool, and another 400
# bytes from the pinned memory pool.  The allocated pinned memory will be
# released just after the transfer is complete.  Note that the actual
# allocation size may be rounded to larger value than the requested size
# for performance.
a = cupy.array(a_cpu)
print(a.nbytes)                          # 400
print(mempool.used_bytes())              # 512
print(mempool.total_bytes())             # 512
print(pinned_mempool.n_free_blocks())    # 1

# When the array goes out of scope, the allocated device memory is released
# and kept in the pool for future reuse.
a = None  # (or `del a`)
print(mempool.used_bytes())              # 0
print(mempool.total_bytes())             # 512
print(pinned_mempool.n_free_blocks())    # 1

# clearing the memory pool by calling `free_all_blocks`.
mempool.free_all_blocks()
pinned_mempool.free_all_blocks()
print(mempool.used_bytes())              # 0
print(mempool.total_bytes())             # 0
print(pinned_mempool.n_free_blocks())    # 0

400
24009216
6000000512
1
400
24009728
6000000512
1
24009216
6000000512
1
24009216
2000000512
0


In [ ]:
# You can also specify the limit in fraction of the total amount of memory
# on the GPU. If you have a GPU with 2 GiB memory, the following is
# equivalent to the above configuration.
#   $ export CUPY_GPU_MEMORY_LIMIT="50%"

import cupy
print(cupy.get_default_memory_pool().get_limit())

mempool = cupy.get_default_memory_pool()

with cupy.cuda.Device(0):
    mempool.set_limit(size=1024**3)  # 1 GiBmempool = cupy.get_default_memory_pool()



### New feature: Stream Ordered memory allocator

 Similar to CuPy’s memory pool, Stream Ordered Memory Allocator also allocates/deallocates memory asynchronously from/to a memory pool in a stream-ordered fashion. The key difference is that it is a built-in feature implemented in the CUDA driver by NVIDIA, so other CUDA applications in the same processs can easily allocate memory from the same pool.

To enable a memory pool that manages stream ordered memory, you can construct a new MemoryAsyncPool instance:

In [ ]:
import cupy

# Use asynchronous stream ordered memory
cupy.cuda.set_allocator(cupy.cuda.MemoryAsyncPool().malloc)

# Create a custom stream
s = cupy.cuda.Stream()

# This would allocate memory asynchronously on stream s
with s:
    a = cupy.empty((100,), dtype=cupy.float64)

Note that if you pass malloc_async() directly to set_allocator() without constructing a MemoryAsyncPool instance, the device’s current memory pool will be used.

When using stream ordered memory, it is important that you maintain a correct stream semantics yourselves using, for example, the Stream and Event APIs (see Streams and Events for details); CuPy does not attempt to act smartly for you. Upon deallocation, the memory is freed asynchronously either on the stream it was allocated (first attempt), or on any current CuPy stream (second attempt). It is permitted that the stream on which the memory was allocated gets destroyed before all memory allocated on it is freed.

In addition, applications/libraries internally use cudaMalloc (CUDA’s default, synchronous allocator) could have unexpected interplay with Stream Ordered Memory Allocator. Specifically, memory freed to the memory pool might not be immediately visible to cudaMalloc, leading to potential out-of-memory errors. In this case, you can either call free_all_blocks() or just manually perform a (event/stream/device) synchronization, and retry.

Currently the MemoryAsyncPool interface is experimental. In particular, while its API is largely identical to that of MemoryPool, several of the pool’s methods require a sufficiently new driver (and of course, a supported hardware, CUDA version, and platform) due to CUDA’s limitation.

. Data copies and kernel launches are enqueued onto the Current Stream, which can be queried via get_current_stream() and changed either by setting up a context manager or by using use().

Events can be created either manually or through the record() method, applied to stream. Event objects can be used for timing GPU activities (via get_elapsed_time()) or setting up inter-stream dependencies:

In [ ]:
#using context manager to switch stream
import numpy as np

a_np = np.arange(10)
s = cp.cuda.Stream()
with s:
    a_cp = cp.asarray(a_np)  # H2D transfer on stream s
    b_cp = cp.sum(a_cp)      # kernel launched on stream s
    assert s == cp.cuda.get_current_stream()

# fall back to the previous stream in use (here the default stream)
# when going out of the scope of s


#using use()

s = cp.cuda.Stream()
s.use()  # any subsequent operations are done on steam s

b_np = cp.asnumpy(b_cp)
assert s == cp.cuda.get_current_stream()
cp.cuda.Stream.null.use()  # fall back to the default (null) stream

assert cp.cuda.Stream.null == cp.cuda.get_current_stream()



e1 = cp.cuda.Event()
e1.record()
a_cp = b_cp * a_cp + 8
e2 = cp.cuda.get_current_stream().record()

# set up a stream order
s2 = cp.cuda.Stream()
s2.wait_event(e2)
with s2:
    # the a_cp is guaranteed updated when this copy (on s2) starts
    a_np = cp.asnumpy(a_cp)

# timing
e2.synchronize()
t = cp.cuda.get_elapsed_time(e1, e2)  # only include the compute time, not the copy time

# User-Defined Kernels
CuPy provides easy ways to define three types of CUDA kernels: elementwise kernels, reduction kernels and raw kernels. In this documentation, we describe how to define and call each kernels.

There exist three main types of user defined kernels:
- elementwise kernel
- reduce kernel
- raw kernel

**Elementwise kernel**

An elementwise kernel can be defined by the ElementwiseKernel class (cp.ElementwiseKernel()). The instance of this class defines a CUDA kernel which can be invoked by the __call__ method of this instance.

A definition of an elementwise kernel consists of four parts:
1. an input argument list (of fixed or generic type)
2. an output argument list
3. a loop body code
4. the kernel name.



In [ ]:
squared_diff = cp.ElementwiseKernel(
   'float32 x, float32 y',
   'float32 z',
   'z = (x - y) * (x - y)',
   'squared_diff')

x = cp.arange(10, dtype=np.float32).reshape(2, 5)
y = cp.random.random(10, dtype=np.float32).reshape(2, 5)
squared_diff(x,y)

#the cupy kernel is able also to perform broadcasting
y = cp.arange(5, dtype=np.float32)
squared_diff(x,y)

### Type-generic kernels
If a type specifier is one character, then it is treated as a type placeholder. It can be used to define a type-generic kernels.

Type placeholders of a same character in the kernel definition indicate the same type. The actual type of these placeholders is determined by the actual argument type. The ElementwiseKernel class first checks the output arguments and then the input arguments to determine the actual type. If no output arguments are given on the kernel invocation, then only the input arguments are used to determine the type.

In [20]:
#Elementwise kernel with different degree of generability

squared_diff_generic = cp.ElementwiseKernel(
    'T x, T y',
    'T z',
    'z = (x - y) * (x - y)',
    'squared_diff_generic')

#We can assume to have the same generic type for input and output
#and we can put the type placeholders in the body of the kernel

squared_diff_more_generic = cp.ElementwiseKernel(
    'T x, T y',
    'T z',
    '''
        T diff = x - y;
        z = diff * diff;
    ''',
    'squared_diff_generic')

#We can assume differen type for different input array and for the output

squared_diff_super_generic = cp.ElementwiseKernel(
    'X x, Y y',
    'Z z',
    'z = (x - y) * (x - y)',
    'squared_diff_super_generic')



array([[7.2708994e-01, 5.7552494e-03, 1.4336106e+00, 5.4559960e+00,
        1.0588921e+01],
       [1.9140984e+01, 2.8782030e+01, 4.7485634e+01, 5.7307125e+01,
        6.4585457e+01]], dtype=float32)

**Reduce kernel**

Reduction kernels can be defined by the ReductionKernel class. We can use it by defining four parts of the kernel code:

1.  Identity value: This value is used for the initial value of reduction.

2.  Mapping expression: It is used for the pre-processing of each element to be reduced.

3.  Reduction expression: It is an operator to reduce the multiple mapped values. The special variables a and b are used for its operands.

4.  Post mapping expression: It is used to transform the resulting reduced values. The special variable a is used as its input. Output should be written to the output parameter.

ReductionKernel class automatically inserts other code fragments that are required for an efficient and flexible reduction implementation.



In [ ]:
#L2 norm along specified axes can be written as follows:

l2norm_kernel = cp.ReductionKernel(
    'T x',  # input params
    'T y',  # output params
    'x * x',  # map
    'a + b',  # reduce
    'y = sqrt(a)',  # post-reduction map
    '0',  # identity value
    'l2norm'  # kernel name
)
x = cp.arange(10, dtype=np.float32).reshape(2, 5)
l2norm_kernel(x, axis=1)


**Raw kernel**

Raw kernels can be defined by the RawKernel class. By using raw kernels, you can define kernels from raw CUDA source.

RawKernel object allows you to call the kernel with CUDA cuLaunchKernel interface. In other words, you have control over grid size, block size, shared memory size and stream.

In [31]:
import cupy as cp
import math

add_kernel = cp.RawKernel(r'''
extern "C" __global__
void my_add(const float* x1, const float* x2, float* y) {
    int tid = blockDim.x * blockIdx.x + threadIdx.x;
    y[tid] = x1[tid] + x2[tid];
}
''', 'my_add')


x1 = cp.arange(25, dtype=cp.float32)
print(x1)
x2 = cp.arange(25, dtype=cp.float32)
y = cp.zeros(25, dtype=cp.float32)
threads_per_block = 512
size=25
grid_size = (int(math.ceil(size / threads_per_block)), 1, 1)
block_size = (threads_per_block, 1, 1)
add_kernel(grid_size, block_size, (x1, x2, y, size))
#add_kernel((5,), (5,), (x1, x2, y))

y

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24.]


array([ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16., 18., 20., 22., 24.,
       26., 28., 30., 32., 34., 36., 38., 40., 42., 44., 46., 48.],
      dtype=float32)

## Kernel fusion

cupy.fuse() is a decorator that fuses functions. This decorator can be used to define an elementwise or reduction kernel more easily than ElementwiseKernel or ReductionKernel.

At the first function call, the fused function analyzes the original function based on the abstracted information of arguments (e.g. their dtypes and ndims) and creates and caches an actual CUDA kernel. From the second function call with the same input types, the fused function calls the previously cached kernel, so it is highly recommended to reuse the same decorated functions instead of decorating local functions that are defined multiple times.

In [ ]:
#By using this decorator, we can define the squared_diff kernel as follows:

@cp.fuse(kernel_name='squared_diff')
def squared_diff(x, y):
    return (x - y) * (x - y)


#cupy.fuse() supports simple reduction kernels
@cp.fuse()(kernel_name='sum_of_products')
def sum_of_products(x, y):
    return cp.sum(x * y, axis = -1)

**JIT kernel definition**

CuPy’s JIT compiler generates CUDA code via Python AST.
The cupyx.jit.rawkernel decorator can create raw CUDA kernels from Python functions.

In this section, a Python function wrapped with the decorator is called a target function.

A target function consists of elementary scalar operations, and users have to manage how to parallelize them. CuPy’s array operations which automatically parallelize operations (e.g., add(), sum()) are not supported. If a custom kernel based on such array functions is desired, please refer to the Kernel fusion section.

In [26]:
from cupyx import jit

@jit.rawkernel()
def elementwise_copy(x, y, size):
    tid = jit.blockIdx.x * jit.blockDim.x + jit.threadIdx.x
    ntid = jit.gridDim.x * jit.blockDim.x
    for i in range(tid, size, ntid):
        y[i] = x[i]

size = cupy.uint32(2 ** 22)
x = cupy.random.normal(size=(size,), dtype=cupy.float32)
y = cupy.empty((size,), dtype=cupy.float32)

elementwise_copy((128,), (1024,), (x, y, size))  # RawKernel style
print(x)
print(y)

assert (x == y).all()

elementwise_copy[128, 1024](x, y, size)  #  Numba style
assert (x == y).all()

[ 1.7807698   0.09125482 -0.75306845 ... -0.7375061  -0.47575653
  0.5274892 ]
[ 1.7807698   0.09125482 -0.75306845 ... -0.7375061  -0.47575653
  0.5274892 ]


/usr/local/lib/python3.10/dist-packages/cupyx/jit/_interface.py:161: FutureWarning: cupyx.jit.rawkernel is experimental. The interface can change in the future.
  cupy._util.experimental('cupyx.jit.rawkernel')


# Exercise 1:

Implement a cuda kernel that, given two input arrays $(x_1, x_2)$ of size N, produces an output array given by

$y[i] = x_1[i]*x_2[i]+ B$

where B is a constant

Time this kernel function using cuda events for different sizes of the input arrays $(10^4, 10^5, 10^6)$
and for different block-grid choices.

Produce a plots with the measured times.

# Exercise 2:

Compare the time of cupy matmul with the matmul obtained with a vanilla CUDA matmul implemented with raw kernels.

